# Transfer öğrenimi (Transfer learning) ve ince ayar (fine-tuning)

Bu çalışma dosyası, önceden eğitilmiş bir ağdan transfer öğrenimini kullanarak kedi ve köpek görüntülerini nasıl sınıflandıracağınızı öğreneceksiniz.

Önceden eğitilmiş bir model, daha önce büyük bir veri kümesinde, genellikle büyük ölçekli bir görüntü sınıflandırma görevinde eğitilmiş kayıtlı bir ağdır. Ya önceden eğitilmiş modeli olduğu gibi kullanırsınız ya da bu modeli belirli bir göreve göre özelleştirmek için transfer öğrenimini kullanırsınız.

Görüntü sınıflandırması için transfer öğreniminin arkasındaki sezgi, bir modelin yeterince geniş ve genel bir veri kümesi üzerinde eğitilmesi durumunda, bu modelin görsel dünyanın genel bir modeli olarak etkin bir şekilde hizmet edeceğidir. Daha sonra, büyük bir modeli büyük bir veri kümesi üzerinde eğiterek sıfırdan başlamak zorunda kalmadan bu öğrenilen özellik haritalarından yararlanabilirsiniz.

Bu çalışma dosyasında, önceden eğitilmiş bir modeli özelleştirmek için iki yol deneyeceksiniz:

1. **Özellik Çıkarma:** Yeni örneklerden anlamlı özellikler çıkarmak için önceki bir ağ tarafından öğrenilen temsilleri kullanın. Önceden eğitilmiş modelin üzerine sıfırdan eğitilecek yeni bir sınıflandırıcı eklemeniz yeterlidir, böylece daha önce öğrenilen özellik haritalarını veri kümesi için yeniden kullanabilirsiniz.

 Tüm modeli (yeniden) eğitmeniz gerekmez. Temel evrişim ağı, resimleri sınıflandırmak için genel olarak yararlı olan özellikleri zaten içerir. Bununla birlikte, önceden eğitilmiş modelin son sınıflandırma kısmı, orijinal sınıflandırma görevine ve daha sonra modelin üzerinde eğitildiği sınıflar kümesine özgüdür.

2. **İnce Ayar:** Donmuş bir model tabanının birkaç üst katmanını çözün ve hem yeni eklenen sınıflandırıcı katmanlarını hem de temel modelin son katmanlarını birlikte eğitin. Bu, belirli görev için onları daha alakalı hale getirmek için temel modeldeki yüksek mertebeden özellik temsillerine "ince ayar yapmamızı" sağlar.

Genel makine öğrenimi iş akışını takip edeceksiniz.

1. Verileri inceleyin ve anlayın
1. Bu durumda Keras ImageDataGenerator kullanarak bir girdi ardışık düzeni oluşturun
1. Modeli oluşturun
   * Önceden eğitilmiş temel modelde (ve önceden eğitilmiş ağırlıklarda) yük
   * Sınıflandırma katmanlarını üste istifleyin
1. Modeli eğitin
1. Modeli değerlendirin

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

## Veri ön işleme

### Veri kümesi indirme

Bu çalışma dosyasında, kedi ve köpeklerin birkaç bin görüntüsünü içeren bir veri kümesi kullanacaksınız. Görüntüleri içeren bir zip dosyasını indirin ve ayıklayın, ardından `tf.keras.utils.image_dataset_from_directory` yardımcı programını kullanarak eğitim ve doğrulama için bir `tf.data.Dataset` oluşturun. [Bu eğitimde](https://www.tensorflow.org/tutorials/load_data/images) resim yükleme hakkında daha fazla bilgi edinebilirsiniz.

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Eğitim kümesindeki ilk dokuz resmi ve etiketi gösterin:

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Orijinal veri kümesi bir test kümesi içermediğinden, bir tane oluşturacaksınız. Bunu yapmak için, `tf.data.experimental.cardinalite`yi kullanarak doğrulama kümesinde kaç tane veri grubu bulunduğunu belirleyin, ardından bunların %20'sini bir test kümesine taşıyın.

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

### Performans için veri kümesini yapılandırın

I/O'yu engellemeden görüntüleri diskten yüklemek için arabelleğe alınmış önceden getirmeyi kullanın. Bu yöntem hakkında daha fazla bilgi edinmek için [veri performansı]((https://www.tensorflow.org/guide/data_performance)) kılavuzuna bakın.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Veri artırmayı kullanın

Büyük bir görüntü veri kümeniz olmadığında, eğitim görüntülerine döndürme ve yatay çevirme gibi rastgele, ancak gerçekçi dönüşümler uygulayarak yapay olarak örnek çeşitliliğini tanıtmak iyi bir uygulamadır. Bu, modeli eğitim verilerinin farklı yönlerine maruz bırakmaya ve [aşırı uydurmayı](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) azaltmaya yardımcı olur. [Bu eğitimde](https://www.tensorflow.org/tutorials/images/data_augmentation) veri büyütme hakkında daha fazla bilgi edinebilirsiniz.

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

Not: Bu katmanlar yalnızca eğitim sırasında `Model.fit`i çağırdığınızda aktiftir. Model, `Model.evaluate` veya `Model.fit` içindeki çıkarım modunda kullanıldığında etkin değildirler.

Bu katmanları tekrar tekrar aynı görüntüye uygulayalım ve sonucu görelim.

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Piksel değerlerini yeniden ölçeklendir

Birazdan, temel modeliniz olarak kullanmak için `tf.keras.applications.MobileNetV2` dosyasını indireceksiniz. Bu model `[-1, 1]` içinde piksel değerleri bekler, ancak bu noktada resimlerinizdeki piksel değerleri `[0, 255]` içindedir. Bunları yeniden ölçeklendirmek için modele dahil olan ön işleme yöntemini kullanın.

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

Not: Alternatif olarak, `tf.keras.layers.Rescaling`i kullanarak piksel değerlerini `[0, 255]` ile `[-1, 1]` arasında yeniden ölçeklendirebilirsiniz.

In [ ]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

Not: Diğer `tf.keras.applications` kullanılıyorsa, `[-1, 1]` veya `[0, 1]` içinde piksel olup olmadığını belirlemek için API belgesini kontrol ettiğinizden veya dahil edilen `preprocess_input`u kullandığınızdan emin olun.

## Önceden eğitilmiş convnet'lerden temel modeli oluşturun
Google'da geliştirilen **MobileNet V2** modelinden temel modeli oluşturacaksınız. Bu, 1,4M görüntü ve 1000 sınıftan oluşan büyük bir veri kümesi olan ImageNet veri kümesinde önceden eğitilmiştir. ImageNet, `jackfruit` ve `syringe` gibi çok çeşitli kategorilere sahip bir araştırma eğitimi veri kümesidir. Bu bilgi temeli, kedileri ve köpekleri belirli veri kümemizden sınıflandırmamıza yardımcı olacaktır.

Öncelikle, özellik çıkarımı için hangi MobileNet V2 katmanını kullanacağınızı seçmeniz gerekir. En son sınıflandırma katmanı (makine öğrenimi modellerinin çoğu diyagramı aşağıdan yukarıya gittiği için "üstte") çok kullanışlı değildir. Bunun yerine, düzleştirme işleminden önceki en son katmana bağlı olmak için yaygın uygulamayı izleyeceksiniz. Bu katmana "darboğaz katmanı (bottleneck layer)" denir. Darboğaz katmanı özellikleri, son/üst katmana kıyasla daha fazla genellik sağlar.

İlk olarak, ImageNet üzerinde eğitilmiş ağırlıklarla önceden yüklenmiş bir MobileNet V2 modelini somutlaştırın. **include_top=False** bağımsız değişkenini belirterek, en üstte sınıflandırma katmanlarını içermeyen bir ağ yüklersiniz, bu özellik çıkarma için idealdir.

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Bu özellik çıkarıcı, her bir `160x160x3` resmi `5x5x1280` özellik bloğuna dönüştürür. Örnek bir toplu resim için ne yaptığını görelim:

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Öznitelik çıkarma
Bu adımda, bir önceki adımdan oluşturulan evrişimsel tabanı donduracak ve bir özellik çıkarıcı olarak kullanacaksınız. Ek olarak, üstüne bir sınıflandırıcı eklersiniz ve üst düzey sınıflandırıcıyı eğitirsiniz.

### Evrişimli tabanı dondurun

Modeli derlemeden ve eğitmeden önce evrişim tabanını dondurmak(freeze) önemlidir. Dondurma (layer.trainable = False ayarlanarak), belirli bir katmandaki ağırlıkların eğitim sırasında güncellenmesini engeller. MobileNet V2'nin birçok katmanı vardır, bu nedenle tüm modelin `(eğitilebilir)trainable` bayrağını False olarak ayarlamak hepsini donduracaktır.

In [ ]:
base_model.trainable = False

### BatchNormalizasyon katmanları hakkında önemli not

Birçok model `tf.keras.layers.BatchNormalization` katmanlarını içerir. Bu katman özel bir durumdur ve bu çalışma dosyasında daha sonra gösterildiği gibi ince ayar bağlamında önlemler alınmalıdır.

`layer.trainable = False` ayarını yaptığınızda, `BatchNormalization` katmanı çıkarım modunda çalışacak ve ortalama ve varyans istatistiklerini güncellemeyecektir.

İnce ayar yapmak için BatchNormalization katmanları içeren bir modeli çözdüğünüzde, temel modeli çağırırken `training = False` ileterek BatchNormalization katmanlarını çıkarım modunda tutmalısınız. Aksi takdirde, eğitilebilir olmayan ağırlıklara uygulanan güncellemeler, modelin öğrendiklerini yok edecektir.

Daha fazla ayrıntı için [Transfer öğrenim kılavuzuna](https://www.tensorflow.org/guide/keras/transfer_learning) bakın.

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Bir sınıflandırma katmanı ekleyin

Özellikler bloğundan tahminler oluşturmak için, özellikleri görüntü başına tek bir 1280 elemanlı vektöre dönüştürmek için bir `tf.keras.layers.GlobalAveragePooling2D` katmanını kullanarak uzamsal(spatial) "5x5" uzamsal konumların ortalamasını alın.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Bu özellikleri görüntü başına tek bir tahmine dönüştürmek için bir `tf.keras.layers.Dense` katmanı uygulayın. Burada bir etkinleştirme işlevine ihtiyacınız yoktur, çünkü bu tahmin bir `logit` veya ham bir tahmin değeri olarak değerlendirilecektir. Pozitif sayılar sınıf 1'i, negatif sayılar sınıf 0'ı öngörür.

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

[Keras Functional API](https://www.tensorflow.org/guide/keras/functional)'sini kullanarak veri arttırma, yeniden ölçeklendirme, `base_model` ve özellik çıkarıcı katmanlarını zincirleyerek bir model oluşturun. Daha önce belirtildiği gibi, modelimiz bir `BatchNormalization` katmanı içerdiğinden `training=False` kullanın.

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

### Modeli derleyin

Modeli eğitmeden önce derleyin. İki sınıf olduğundan, model doğrusal bir çıktı sağladığı için `from_logits=True` ile `tf.keras.losses.BinaryCrossentropy` kaybını kullanın.

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

MobileNet'teki 2,5 milyon parametre donduruldu, ancak Dense katmanda 1,2 bin _eğitilebilir(trainable)_ parametre var. Bunlar, ağırlıklar ve önyargılar olmak üzere iki `tf.Variable` nesnesi arasında bölünür.

In [ ]:
len(model.trainable_variables)

### Modeli eğitin

10 dönemlik eğitimden sonra, doğrulama kümesinde ~%94 doğruluk görmelisiniz.

In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

### Öğrenme eğrileri (Learning curves)

MobileNetV2 temel modelini sabit bir öznitelik çıkarıcı olarak kullanırken eğitim ve doğrulama kümesinin doğruluğu/kaybının (accuracy/loss) öğrenme eğrilerine bir göz atalım.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Not: Doğrulama metriklerinin neden eğitim metriklerinden açıkça daha iyi olduğunu merak ediyorsanız, ana faktör `tf.keras.layers.BatchNormalization` ve `tf.keras.layers.Dropout` gibi katmanların eğitim sırasında doğruluğu etkilemesidir. Doğrulama kaybı hesaplanırken kapatılırlar.

Daha az ölçüde, bunun nedeni aynı zamanda eğitim metriklerinin bir devir(epoch) ortalamasını raporlaması ve doğrulama metriklerinin devirden(epoch) sonra değerlendirilmesidir, dolayısıyla doğrulama metriklerinin biraz daha uzun eğitilmiş bir model görmesidir.

## İnce ayar (Fine tuning)
Öznitelik çıkarma deneyinde, bir MobileNetV2 temel modelinin üzerinde yalnızca birkaç katmanı eğitiyordunuz. Önceden eğitilmiş ağın ağırlıkları, eğitim sırasında **güncellenmedi**.

Performansı daha da artırmanın bir yolu, eklediğiniz sınıflandırıcının eğitiminin yanı sıra önceden eğitilmiş modelin üst katmanlarının ağırlıklarını eğitmek (veya "ince ayar yapmak"). Eğitim süreci, ağırlıkların genel öznitelik haritalarından özellikle veri kümesiyle ilişkilendirilen özelliklere ayarlanmasını zorlayacaktır.

Not: Bu, yalnızca önceden eğitilmiş model eğitilemez olarak ayarlanmış olarak üst düzey sınıflandırıcıyı eğittikten sonra denenmelidir. Önceden eğitilmiş bir modelin üzerine rastgele başlatılmış bir sınıflandırıcı ekler ve tüm katmanları birlikte eğitmeye çalışırsanız, degrade güncellemelerinin büyüklüğü (sınıflandırıcıdan gelen rastgele ağırlıklar nedeniyle) çok büyük olur ve önceden eğitilmiş modeliniz öğrendiklerini unut.

Ayrıca, tüm MobileNet modeli yerine az sayıda üst katmanda ince ayar yapmaya çalışmalısınız. Çoğu evrişimsel ağda, bir katman ne kadar yüksekse, o kadar özelleşir. İlk birkaç katman, hemen hemen tüm görüntü türlerini genelleştiren çok basit ve genel özellikleri öğrenir. Siz yukarı çıktıkça, özellikler modelin üzerinde eğitildiği veri kümesine giderek daha özel hale gelir. İnce ayarın amacı, genel öğrenmenin üzerine yazmak yerine bu özelleştirilmiş özellikleri yeni veri kümesiyle çalışacak şekilde uyarlamaktır.

### Modelin üst katmanlarını çözdürün (Un-freeze)

Tek yapmanız gereken `base_model`i çözmek ve alt katmanları eğitilemez hale getirmek. Ardından, modeli yeniden derlemelisiniz (bu değişikliklerin etkili olması için gerekli) ve eğitime devam etmelisiniz.

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
# bu katmandan itibaren ince ayar yapılacak
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Modeli derleyin

Çok daha büyük bir modeli eğittiğiniz ve önceden eğitilmiş ağırlıkları yeniden uyarlamak istediğiniz için bu aşamada daha düşük bir öğrenme oranı kullanmak önemlidir. Aksi takdirde, modeliniz çok hızlı bir şekilde aşırı uyuma yol açabilir.

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Modeli eğitmeye devam edin

Daha önce yakınsama(convergence) eğitimi aldıysanız, bu adım doğruluğunuzu birkaç yüzde puanı artıracaktır.

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Şimdi MobileNetV2 temel modelinin son birkaç katmanına ince ayar yaparken ve bunun üzerine sınıflandırıcıyı eğitirken eğitim ve doğrulama kümesinin doğruluğu/kaybının öğrenme eğrilerine bir göz atalım. Doğrulama kaybı, eğitim kaybından çok daha yüksektir, bu nedenle biraz aşırı uyum sağlayabilirsiniz.

Yeni eğitim kümesi nispeten küçük olduğundan ve orijinal MobileNetV2 veri kümesine benzer olduğundan, biraz aşırı uyum sağlayabilirsiniz.

İnce ayardan sonra model, doğrulama kümesinde neredeyse %98 doğruluğa ulaşır.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Değerlendirme ve tahmin

Son olarak, test kümesini kullanarak modelin performansını yeni veriler üzerinde doğrulayabilirsiniz.

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

Artık evcil hayvanınızın kedi mi yoksa köpek mi olduğunu tahmin etmek için bu modeli kullanmaya hazırsınız.

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## Özet

* **Özellik çıkarma için önceden eğitilmiş bir model kullanma**: Küçük bir veri kümesiyle çalışırken, aynı etki alanındaki daha büyük bir veri kümesinde eğitilmiş bir model tarafından öğrenilen özelliklerden yararlanmak yaygın bir uygulamadır. Bu, önceden eğitilmiş modeli somutlaştırarak ve üstüne tam bağlantılı bir sınıflandırıcı ekleyerek yapılır. Önceden eğitilmiş model "dondurulur" ve eğitim sırasında yalnızca sınıflandırıcının ağırlıkları güncellenir.
Bu durumda, evrişimli taban, her bir görüntüyle ilişkili tüm öznitelikleri çıkardı ve siz az önce, çıkarılan öznitelikler kümesi verilen görüntü sınıfını belirleyen bir sınıflandırıcıyı eğittiniz.

* **Önceden eğitilmiş bir modelde ince ayar yapma**: Performansı daha da artırmak için, önceden eğitilmiş modellerin üst düzey katmanlarını ince ayar yoluyla yeni veri kümesine yeniden kullanmak isteyebilirsiniz.
Bu durumda, ağırlıklarınızı, modelinizin veri kümesine özgü üst düzey özellikleri öğreneceği şekilde ayarladınız. Bu teknik genellikle eğitim veri kümesi büyük olduğunda ve önceden eğitilmiş modelin üzerinde eğitildiği orijinal veri kümesine çok benzer olduğunda önerilir.

Daha fazla bilgi edinmek için [Transfer öğrenim kılavuzunu](https://www.tensorflow.org/guide/keras/transfer_learning) ziyaret edin.
